In [8]:
import tensorflow as tf
import gym
import numpy as np

tf.reset_default_graph()

n_inputs = 4
n_hidden = 4
n_outputs = 1

learning_rate = 0.01

initializer = tf.variance_scaling_initializer()

X = tf.placeholder(tf.float32, shape=[None, n_inputs])

hidden = tf.layers.dense(X, n_hidden, activation=tf.nn.elu, kernel_initializer=initializer)
logits = tf.layers.dense(hidden, n_outputs)
outputs = tf.nn.sigmoid(logits)  # probability of action 0 (left)
p_left_and_right = tf.concat(axis=1, values=[outputs, 1 - outputs])
action = tf.multinomial(tf.log(p_left_and_right), num_samples=1)

y = 1. - tf.to_float(action)
cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits)
optimizer = tf.train.AdamOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(cross_entropy)
gradients = [grad for grad, variable in grads_and_vars]
gradient_placeholders = []
grads_and_vars_feed = []
for grad, variable in grads_and_vars:
    print(grad)
    gradient_placeholder = tf.placeholder(tf.float32, shape=grad.get_shape())
    gradient_placeholders.append(gradient_placeholder)
    grads_and_vars_feed.append((gradient_placeholder, variable))
training_op = optimizer.apply_gradients(grads_and_vars_feed)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

print(gradient_placeholders)

def discount_rewards(rewards, discount_rate):
    discounted_rewards = np.zeros(len(rewards))
    cumulative_rewards = 0
    for step in reversed(range(len(rewards))):
        cumulative_rewards = rewards[step] + cumulative_rewards * discount_rate
        discounted_rewards[step] = cumulative_rewards
    return discounted_rewards

def discount_and_normalize_rewards(all_rewards, discount_rate):
    all_discounted_rewards = [discount_rewards(rewards, discount_rate) for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted_rewards - reward_mean)/reward_std for discounted_rewards in all_discounted_rewards]

Tensor("gradients/dense/MatMul_grad/tuple/control_dependency_1:0", shape=(4, 4), dtype=float32)
Tensor("gradients/dense/BiasAdd_grad/tuple/control_dependency_1:0", shape=(4,), dtype=float32)
Tensor("gradients/dense_1/MatMul_grad/tuple/control_dependency_1:0", shape=(4, 1), dtype=float32)
Tensor("gradients/dense_1/BiasAdd_grad/tuple/control_dependency_1:0", shape=(1,), dtype=float32)
[<tf.Tensor 'Placeholder_1:0' shape=(4, 4) dtype=float32>, <tf.Tensor 'Placeholder_2:0' shape=(4,) dtype=float32>, <tf.Tensor 'Placeholder_3:0' shape=(4, 1) dtype=float32>, <tf.Tensor 'Placeholder_4:0' shape=(1,) dtype=float32>]


In [9]:
env = gym.make("CartPole-v0")

n_games_per_update = 10
n_max_steps = 1000
save_iterations = 10
discount_rate = 0.95

with tf.Session() as sess:
    init.run()
    
    all_rewards = []
    all_gradients = []
    
    
    current_rewards = []
    current_gradients = []
    obs = env.reset()
    for step in range(n_max_steps):
        action_val, gradients_val = sess.run([action, gradients], feed_dict={X: obs.reshape(1, n_inputs)})
        obs, reward, done, info = env.step(action_val[0][0])
        
        # print(gradients_val)
        current_rewards.append(reward)
        current_gradients.append(gradients_val)
        #print(":",gradients_val[0])
        #print("::",gradients_val[1])
        #print(":::",gradients_val[2])
        #print("::::",gradients_val[3])
        if done:
            break
            
    
        
    all_rewards.append(current_rewards)
    all_gradients.append(current_gradients)


    all_rewards = discount_and_normalize_rewards(all_rewards, discount_rate=discount_rate)
    
    
    feed_dict = {}
    for var_index, gradient_placeholder in enumerate(gradient_placeholders):
        mean_gradients = np.mean([reward * all_gradients[game_index][step][var_index]
                                  for game_index, rewards in enumerate(all_rewards)
                                  for step, reward in enumerate(rewards)], axis=0)
        feed_dict[gradient_placeholder] = mean_gradients
    sess.run(training_op, feed_dict=feed_dict)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
.
.
.
.


In [26]:
discount_and_normalize_rewards([[10, 0, -50],[-50],[50],[-10,0,50]], discount_rate=0.8)

[array([-0.5227733 , -0.95049691, -1.18812114]),
 array([-1.18812114]),
 array([1.18812114]),
 array([0.5227733 , 0.95049691, 1.18812114])]

In [52]:
np.mean([
        [
            [np.array([2.,3.]), np.array([4.,5.])], 
            [np.array([1.,2.]), np.array([3.,4.])]
        ],
        [
            [np.array([3.,4.]), np.array([5.,6.])],
            [np.array([1.,2.]), np.array([3.,4.])]
        ]
        ], axis=0)

array([[[2.5, 3.5],
        [4.5, 5.5]],

       [[1. , 2. ],
        [3. , 4. ]]])

In [93]:
b=[]

a=[]
a.append([np.array([1.,2.,7.]), np.array([7.,8.])])
a.append([np.array([3.,4.,8.]), np.array([3.,5.])])

b.append(a)

a=[]
a.append([np.array([7.,9.,10.]), np.array([7.,8.])])
a.append([np.array([3.,9.,11.]), np.array([3.,5.])])

b.append(a)

b

[[[array([1., 2., 7.]), array([7., 8.])],
  [array([3., 4., 8.]), array([3., 5.])]],
 [[array([ 7.,  9., 10.]), array([7., 8.])],
  [array([ 3.,  9., 11.]), array([3., 5.])]]]

In [112]:
c=[1 - 1 + b[i][step][0]
   for i, grads in enumerate(b)
    for step, grad in enumerate(grads)]
c

[array([1., 2., 7.]),
 array([3., 4., 8.]),
 array([ 7.,  9., 10.]),
 array([ 3.,  9., 11.])]

In [113]:
np.mean(c, axis=0)

array([3.5, 6. , 9. ])